In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gzip
import json

import pandas as pd

from matpowercaseframes import CaseFrames

## UnitCommitment.jl Data

In [3]:
CASE_NAME = "../data/case14/2017-01-01.json.gz"
with gzip.open(CASE_NAME, "rt", encoding="utf-8") as f:
    data = json.load(f)

In [4]:
# TODO: convert `UnitCommitment.jl` format to matpowercaseframes format
data.keys()

dict_keys(['SOURCE', 'Parameters', 'Generators', 'Transmission lines', 'Contingencies', 'Buses', 'Reserves'])

## MATPOWER Data

In [5]:
CASE_NAME = "case14.m"
cf_14 = CaseFrames(CASE_NAME)

In [6]:
# Helper functions for data processing
def _extract_scalar_bounds(points):
    """Extract min/max from possibly nested array structure for time-varying limits."""
    if not isinstance(points, list):
        return points, points
    if len(points) == 0:
        return 0.0, 0.0
    # Handle nested arrays like [5.0, [10.0, 12.0, 15.0, 20.0]]
    flat = []
    for p in points:
        if isinstance(p, list):
            flat.extend(p)
        else:
            flat.append(p)
    return min(flat), max(flat)


def _get_last_value(val):
    """Get last value from time-series or return scalar."""
    if isinstance(val, list) and len(val) > 0:
        return val[-1]
    return val


def _extract_parameters(parameters: dict) -> dict:
    """Extract and set default values for system parameters."""
    version = parameters.get("Version", "0.0")
    return {
        "version": f"UnitCommitment.jl-{version}",
        "time_horizon_min": parameters.get("Time horizon (min)"),
        "time_horizon_h": parameters.get("Time horizon (h)"),
        "time_step_min": parameters.get("Time step (min)", 60),
        "power_balance_penalty": parameters.get("Power balance penalty ($/MW)", 1000.0),
        "scenario_name": parameters.get("Scenario name", "s1"),
        "scenario_weight": parameters.get("Scenario weight", 1.0),
    }


def _infer_generator_type(gen_data: dict) -> str:
    """
    Infer generator type from data.

    v0.4+: Use explicit "Type" field
    v0.3: Infer from presence of "Production cost curve (MW)"
    """
    if "Type" in gen_data:
        return gen_data["Type"]
    # Version 0.3: Infer type
    if "Production cost curve (MW)" in gen_data:
        return "Thermal"
    return "Profiled"


def _process_profiled_generator(gen_name: str, gen_data: dict, gen_bus: str) -> tuple:
    """Process a profiled (renewable/hydro) generator."""
    cost_per_mw = gen_data["Cost ($/MW)"]  # Required
    min_power = gen_data.get("Minimum power (MW)", 0.0)  # Default: 0.0
    max_power = gen_data["Maximum power (MW)"]  # Required

    matpower_gen = {
        "GEN_NAME": gen_name,
        "GEN_BUS": gen_bus,
        "TYPE": "Profiled",
        "PG": 0.0,
        "QG": 0.0,
        "QMAX": 0.0,
        "QMIN": 0.0,
        "VG": 1.0,
        "MBASE": 100.0,
        "GEN_STATUS": 1,
        "PMAX": max_power,
        "PMIN": min_power,
        "COST_PER_MW": cost_per_mw,
        "MIN_POWER": min_power,
        "MAX_POWER": max_power,
    }

    matpower_gencost = {
        "MODEL": 1,
        "STARTUP": 0.0,
        "SHUTDOWN": 0.0,
        "NCOST": 2,
        "X1": min_power,
        "Y1": min_power * cost_per_mw,
        "X2": max_power,
        "Y2": max_power * cost_per_mw,
    }

    return matpower_gen, matpower_gencost


def _process_thermal_generator(gen_name: str, gen_data: dict, gen_bus: str) -> tuple:
    """Process a thermal (coal/gas/nuclear) generator."""
    # Required fields
    mw_points = gen_data["Production cost curve (MW)"]
    cost_points = gen_data["Production cost curve ($)"]
    initial_status = gen_data["Initial status (h)"]
    initial_power = gen_data["Initial power (MW)"]

    pmin, pmax = _extract_scalar_bounds(mw_points)
    gen_status = 1 if initial_status > 0 else 0

    # Ramp limits with defaults
    ramp_up = gen_data.get("Ramp up limit (MW)", float("inf"))
    ramp_down = gen_data.get("Ramp down limit (MW)", float("inf"))

    matpower_gen = {
        "GEN_NAME": gen_name,
        "GEN_BUS": gen_bus,
        "TYPE": "Thermal",
        "PG": initial_power,
        "QG": 0.0,
        "QMAX": 0.0,
        "QMIN": 0.0,
        "VG": 1.0,
        "MBASE": 100.0,
        "GEN_STATUS": gen_status,
        "PMAX": pmax,
        "PMIN": pmin,
        "RAMP_UP": ramp_up,
        "RAMP_DOWN": ramp_down,
        "RAMP_UP_SU": gen_data.get("Startup limit (MW)", float("inf")),
        "RAMP_DOWN_SD": gen_data.get("Shutdown limit (MW)", float("inf")),
        "MUT": gen_data.get("Minimum uptime (h)", 1),
        "MDT": gen_data.get("Minimum downtime (h)", 1),
        "INITIAL_STATUS": initial_status,
        "INITIAL_POWER": initial_power,
        "MUST_RUN": gen_data.get("Must run?", False),
        "RESERVE_ELIGIBILITY": gen_data.get("Reserve eligibility", []),
        "COMMITMENT_STATUS": gen_data.get("Commitment status"),
    }

    # Compute MATPOWER ramp rates
    if ramp_up != float("inf"):
        matpower_gen["RAMP_AGC"] = ramp_up / 60.0
        matpower_gen["RAMP_10"] = ramp_up / 6.0
        matpower_gen["RAMP_30"] = ramp_up / 2.0

    # Gencost with startup costs
    startup_costs = gen_data.get("Startup costs ($)", [0.0])
    startup_delays = gen_data.get("Startup delays (h)", [1])

    matpower_gencost = {
        "MODEL": 1,
        "STARTUP": startup_costs[0] if len(startup_costs) > 0 else 0.0,
        "SHUTDOWN": 0.0,
        "NCOST": len(mw_points) if isinstance(mw_points, list) else 1,
    }

    # Store all startup cost tiers
    for i, (delay, cost) in enumerate(zip(startup_delays, startup_costs), start=1):
        matpower_gencost[f"STARTUP_DELAY_{i}"] = delay
        matpower_gencost[f"STARTUP_COST_{i}"] = cost

    # Production cost curve points
    if isinstance(mw_points, list):
        for i, (mw, cost) in enumerate(zip(mw_points, cost_points), start=1):
            matpower_gencost[f"X{i}"] = mw
            matpower_gencost[f"Y{i}"] = cost

    return matpower_gen, matpower_gencost


def _process_generators(data: dict) -> tuple:
    """Process all generators and return gen and gencost DataFrames."""
    gen_list = []
    gencost_list = []

    for gen_name, gen_data in data.get("Generators", {}).items():
        gen_type = _infer_generator_type(gen_data)
        gen_bus = gen_data["Bus"]  # Required

        if gen_type == "Profiled":
            gen, gencost = _process_profiled_generator(gen_name, gen_data, gen_bus)
        else:  # Thermal
            gen, gencost = _process_thermal_generator(gen_name, gen_data, gen_bus)

        gen_list.append(gen)
        gencost_list.append(gencost)

    # Define minimum columns for empty DataFrames
    GEN_COLS = ["GEN_NAME", "GEN_BUS", "TYPE", "PG", "GEN_STATUS", "PMAX", "PMIN"]
    GENCOST_COLS = ["MODEL", "STARTUP", "SHUTDOWN", "NCOST"]

    gen_df = pd.DataFrame(gen_list) if gen_list else pd.DataFrame(columns=GEN_COLS)
    gencost_df = (
        pd.DataFrame(gencost_list)
        if gencost_list
        else pd.DataFrame(columns=GENCOST_COLS)
    )

    return gen_df, gencost_df


def _process_buses(data: dict) -> pd.DataFrame:
    """Process all buses and return DataFrame."""
    bus_list = []

    for bus_name, bus_data in data.get("Buses", {}).items():
        matpower_bus = {
            "BUS_I": bus_name,
            "BUS_NAME": bus_name,
            "BUS_TYPE": 1,
            "PD": bus_data["Load (MW)"],  # Required
            "QD": 0.0,
            "GS": 0.0,
            "BS": 0.0,
            "BUS_AREA": 1,
            "VM": 1.0,
            "VA": 0.0,
            "BASE_KV": 1.0,
            "ZONE": 1,
            "VMAX": 1.1,
            "VMIN": 0.9,
        }
        bus_list.append(matpower_bus)

    BUS_COLS = ["BUS_I", "BUS_NAME", "BUS_TYPE", "PD"]
    return pd.DataFrame(bus_list) if bus_list else pd.DataFrame(columns=BUS_COLS)


def _process_storage(data: dict) -> pd.DataFrame:
    """Process all storage units and return DataFrame."""
    storage_list = []

    for storage_name, storage_data in data.get("Storage units", {}).items():
        min_level = storage_data.get("Minimum level (MWh)", 0.0)
        max_level = storage_data["Maximum level (MWh)"]  # Required

        matpower_storage = {
            "STORAGE_NAME": storage_name,
            "STORAGE_BUS": storage_data["Bus"],  # Required
            "MIN_LEVEL_MWH": min_level,
            "MAX_LEVEL_MWH": max_level,
            "ALLOW_SIMULTANEOUS": storage_data.get(
                "Allow simultaneous charging and discharging", True
            ),
            "CHARGE_COST": storage_data["Charge cost ($/MW)"],  # Required
            "DISCHARGE_COST": storage_data["Discharge cost ($/MW)"],  # Required
            "CHARGE_EFF": storage_data.get("Charge efficiency", 1.0),
            "DISCHARGE_EFF": storage_data.get("Discharge efficiency", 1.0),
            "LOSS_FACTOR": storage_data.get("Loss factor", 0.0),
            "MIN_CHARGE_RATE": storage_data.get("Minimum charge rate (MW)", 0.0),
            "MAX_CHARGE_RATE": storage_data["Maximum charge rate (MW)"],  # Required
            "MIN_DISCHARGE_RATE": storage_data.get("Minimum discharge rate (MW)", 0.0),
            "MAX_DISCHARGE_RATE": storage_data[
                "Maximum discharge rate (MW)"
            ],  # Required
            "INIT_LEVEL": storage_data.get("Initial level (MWh)", 0.0),
            "LAST_MIN_LEVEL": storage_data.get(
                "Last period minimum level (MWh)", _get_last_value(min_level)
            ),
            "LAST_MAX_LEVEL": storage_data.get(
                "Last period maximum level (MWh)", _get_last_value(max_level)
            ),
        }
        storage_list.append(matpower_storage)

    STORAGE_COLS = [
        "STORAGE_NAME",
        "STORAGE_BUS",
        "MAX_LEVEL_MWH",
        "MAX_CHARGE_RATE",
        "MAX_DISCHARGE_RATE",
        "CHARGE_COST",
        "DISCHARGE_COST",
    ]
    return (
        pd.DataFrame(storage_list)
        if storage_list
        else pd.DataFrame(columns=STORAGE_COLS)
    )


def _process_price_sensitive_loads(data: dict) -> pd.DataFrame:
    """Process all price-sensitive loads and return DataFrame."""
    ps_load_list = []

    for ps_load_name, ps_load_data in data.get("Price-sensitive loads", {}).items():
        matpower_ps_load = {
            "PS_LOAD_NAME": ps_load_name,
            "PS_LOAD_BUS": ps_load_data["Bus"],  # Required
            "REVENUE": ps_load_data["Revenue ($/MW)"],  # Required
            "DEMAND": ps_load_data["Demand (MW)"],  # Required
        }
        ps_load_list.append(matpower_ps_load)

    PS_LOAD_COLS = ["PS_LOAD_NAME", "PS_LOAD_BUS", "REVENUE", "DEMAND"]
    return (
        pd.DataFrame(ps_load_list)
        if ps_load_list
        else pd.DataFrame(columns=PS_LOAD_COLS)
    )


def _process_transmission_lines(data: dict) -> pd.DataFrame:
    """Process all transmission lines and return DataFrame."""
    branch_list = []

    for line_name, line_data in data.get("Transmission lines", {}).items():
        susceptance = line_data["Susceptance (S)"]  # Required
        matpower_branch = {
            "LINE_NAME": line_name,
            "F_BUS": line_data["Source bus"],  # Required
            "T_BUS": line_data["Target bus"],  # Required
            "BR_R": 0.0,
            "BR_X": 1.0 / susceptance if susceptance != 0 else 0.0,
            "BR_B": susceptance,
            "RATE_A": line_data.get("Normal flow limit (MW)", float("inf")),
            "RATE_B": line_data.get("Emergency flow limit (MW)", float("inf")),
            "RATE_C": line_data.get("Emergency flow limit (MW)", float("inf")),
            "TAP": 0.0,
            "SHIFT": 0.0,
            "BR_STATUS": 1,
            "ANGMIN": -360.0,
            "ANGMAX": 360.0,
            "FLOW_LIMIT_PENALTY": line_data.get("Flow limit penalty ($/MW)", 5000.0),
        }
        branch_list.append(matpower_branch)

    BRANCH_COLS = ["LINE_NAME", "F_BUS", "T_BUS", "BR_B"]
    return (
        pd.DataFrame(branch_list) if branch_list else pd.DataFrame(columns=BRANCH_COLS)
    )


def _process_reserves(data: dict) -> pd.DataFrame:
    """Process all reserves and return DataFrame."""
    reserve_list = []

    for reserve_name, reserve_data in data.get("Reserves", {}).items():
        matpower_reserve = {
            "RESERVE_NAME": reserve_name,
            "TYPE": reserve_data["Type"],  # Required
            "AMOUNT_MW": reserve_data["Amount (MW)"],  # Required
            "SHORTFALL_PENALTY": reserve_data.get("Shortfall penalty ($/MW)", -1),
        }
        reserve_list.append(matpower_reserve)

    RESERVE_COLS = ["RESERVE_NAME", "TYPE", "AMOUNT_MW"]
    return (
        pd.DataFrame(reserve_list)
        if reserve_list
        else pd.DataFrame(columns=RESERVE_COLS)
    )


def _process_contingencies(data: dict) -> pd.DataFrame:
    """Process all contingencies and return DataFrame."""
    contingency_list = []

    for cont_name, cont_data in data.get("Contingencies", {}).items():
        matpower_contingency = {
            "CONTINGENCY_NAME": cont_name,
            "AFFECTED_GENERATORS": cont_data.get("Affected generators", []),
            "AFFECTED_LINES": cont_data.get("Affected lines", []),
        }
        contingency_list.append(matpower_contingency)

    CONTINGENCY_COLS = ["CONTINGENCY_NAME", "AFFECTED_GENERATORS", "AFFECTED_LINES"]
    return (
        pd.DataFrame(contingency_list)
        if contingency_list
        else pd.DataFrame(columns=CONTINGENCY_COLS)
    )


# TODO: apply to matpowercaseframes
def unitcommitmentjl_to_matpower(data: dict):
    """
    Convert UnitCommitment.jl JSON format to MATPOWER CaseFrames format.

    Args:
        data: Dictionary loaded from UnitCommitment.jl JSON file

    Returns:
        Dictionary with 'info' and DataFrames for each component type
    """
    # Extract and validate parameters
    info = _extract_parameters(data.get("Parameters", {}))

    # Process each component type
    gen_df, gencost_df = _process_generators(data)
    bus_df = _process_buses(data)
    storage_df = _process_storage(data)
    ps_load_df = _process_price_sensitive_loads(data)
    branch_df = _process_transmission_lines(data)
    reserve_df = _process_reserves(data)
    contingency_df = _process_contingencies(data)

    return {
        "info": info,
        "gen": gen_df,
        "gencost": gencost_df,
        "bus": bus_df,
        "storage": storage_df,
        "ps_load": ps_load_df,
        "branch": branch_df,
        "reserve": reserve_df,
        "contingency": contingency_df,
    }

In [7]:
dfs = unitcommitmentjl_to_matpower(data)
for attribute, df in dfs.items():
    display(attribute, df)

'info'

{'version': 'UnitCommitment.jl-0.3',
 'time_horizon_min': None,
 'time_horizon_h': 36,
 'time_step_min': 60,
 'power_balance_penalty': 1000.0,
 'scenario_name': 's1',
 'scenario_weight': 1.0}

'gen'

,GEN_NAME,GEN_BUS,TYPE,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,...,MUT,MDT,INITIAL_STATUS,INITIAL_POWER,MUST_RUN,RESERVE_ELIGIBILITY,COMMITMENT_STATUS,RAMP_AGC,RAMP_10,RAMP_30
0,g1,b1,Thermal,230.74888,0.0,0.0,0.0,1.0,100.0,1,...,1,1,24,230.74888,False,[r1],None,3.852000,38.520000,115.560
1,g2,b2,Thermal,0.00000,0.0,0.0,0.0,1.0,100.0,0,...,1,1,-24,0.00000,False,[r1],None,1.543667,15.436667,46.310
2,g3,b3,Thermal,0.00000,0.0,0.0,0.0,1.0,100.0,0,...,1,1,-24,0.00000,False,[r1],None,1.103500,11.035000,33.105
3,g4,b6,Thermal,0.00000,0.0,0.0,0.0,1.0,100.0,0,...,4,4,-24,0.00000,False,[r1],None,1.115667,11.156667,33.470
4,g5,b8,Thermal,0.00000,0.0,0.0,0.0,1.0,100.0,0,...,1,1,-24,0.00000,False,[r1],None,1.084000,10.840000,32.520


'gencost'

...

,LINE_NAME,F_BUS,T_BUS,BR_R,BR_X,BR_B,RATE_A,RATE_B,RATE_C,TAP,SHIFT,BR_STATUS,ANGMIN,ANGMAX,FLOW_LIMIT_PENALTY
0,l1,b1,b2,0.0,0.033902,29.49686,inf,inf,inf,0.0,0.0,1,-360.0,360.0,5000.0
1,l2,b1,b5,0.0,0.127793,7.82518,inf,inf,inf,0.0,0.0,1,-360.0,360.0,5000.0
2,l3,b2,b3,0.0,0.113428,8.81613,inf,inf,inf,0.0,0.0,1,-360.0,360.0,5000.0
3,l4,b2,b4,0.0,0.101024,9.89865,inf,inf,inf,0.0,0.0,1,-360.0,360.0,5000.0
4,l5,b2,b5,0.0,0.099626,10.03755,inf,inf,inf,0.0,0.0,1,-360.0,360.0,5000.0
5,l6,b3,b4,0.0,0.097993,10.20481,inf,inf,inf,0.0,0.0,1,-360.0,360.0,5000.0
6,l7,b4,b5,0.0,0.024127,41.44691,inf,inf,inf,0.0,0.0,1,-360.0,360.0,5000.0
7,l8,b4,b7,0.0,0.119817,8.34607,inf,inf,inf,0.0,0.0,1,-360.0,360.0,5000.0
8,l9,b4,b9,0.0,0.318667,3.13807,inf,inf,inf,0.0,0.0,1,-360.0,360.0,5000.0
9,l10,b5,b6,0.0,0.144397,6.92536,inf,inf,inf,0.0,0.0,1,-360.0,360.0,5000.0


'reserve'

,RESERVE_NAME,TYPE,AMOUNT_MW,SHORTFALL_PENALTY
0,r1,Spinning,"[4.61, 4.43, 4.31, 4.24, 4.29, 4.38, 4.62, 4.8...",-1


'contingency'

,CONTINGENCY_NAME,AFFECTED_GENERATORS,AFFECTED_LINES
0,c1,[],[l1]
1,c2,[],[l2]
2,c3,[],[l3]
3,c4,[],[l4]
4,c5,[],[l5]
5,c6,[],[l6]
6,c7,[],[l7]
7,c8,[],[l8]
8,c9,[],[l9]
9,c10,[],[l10]
